In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/player-stats/all-teams/45?conference=11&division=-1&playertype=skater&position=skaters&rookie=no&sort=points&statstype=standard&page=1&league=1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for i, row in enumerate(rows[1:], start=1):  
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for i, ele in enumerate(cols) if i != 0 and i != 2 and i != len(cols) - 1]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Jersey#', 'Name', 'Pos', 'Division', 'Team', 'GP', 'G', 'A', 'PTS', 'Pt/G', 'PPG', 'SHG', 'GWG', 'SOGW', 'PIM', 'SH%']

df = pd.DataFrame(acha_dynamic_stats, columns=column_names)

# Skip every other row, keeping only odd-indexed rows (1, 3, 5, etc.)
df = df.iloc[::2]
df = df[df['GP'].notna()]

numeric_columns = ['Jersey#', 'GP', 'G', 'A', 'PTS', 'Pt/G', 'PPG', 'SHG', 'GWG', 'SOGW', 'PIM', 'SH%']
for column in numeric_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')  

string_columns = ['Name', 'Pos', 'Division', 'Team']
for column in string_columns:
    df[column] = df[column].astype(str)
    


print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_wd1_skaters.csv', index=False)

      Jersey#               Name Pos   Division                       Team  \
0         9.0       Loki Antonio   F   WD1 WMCH  WD1 Lindenwood University   
2        14.0     Madison Teague   D   WD1 WMCH  WD1 Lindenwood University   
4        17.0    Ardyn Hawryshko   F   WD1 WMCH  WD1 Lindenwood University   
6        16.0      Tess McKerrow   F  WD1 CCWHA         WD1 Adrian College   
8        13.0         Aria Groot   D   WD1 WMCH   WD1 McKendree University   
...       ...                ...  ..        ...                        ...   
1650     21.0  Tanna Christensen   D   WD1 WMCH     WD1 Midland University   
1652     76.0      Davanna Ditto   D   WD1 WMCH     WD1 Midland University   
1654     26.0   Kirsten Martinez   D   WD1 WMCH  WD1 Lindenwood University   
1656     16.0      Lauren Miller   F   WD1 WMCH  WD1 Lindenwood University   
1658      7.0      Hailey Putnam   D   WD1 WMCH   WD1 Maryville University   

      GP   G   A  PTS  Pt/G  PPG  SHG  GWG  SOGW   PIM  SH%  
0